In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import csv
import numpy as np
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/bigPatentData.csv", header=None)
df = df.sample(frac=1).reset_index(drop=True)
app_num = df[0]
raw_text = df[3]
y = df[4]

In [ ]:
train_appno = np.array(app_num[:1350])
test_appno = np.array(app_num[1350:])
train_y = np.array(y[:1350])
test_y = np.array(y[1350:])

In [ ]:
print(np.unique(train_y, return_counts=True))

(array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'y'], dtype=object), array([143, 149, 146, 149, 151, 152, 156, 158, 146]))


In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import string

stop_words = set(stopwords.words('english'))
def remove_stopwords(data):
  new_data = []
  for w in data:
    if w not in stop_words:
      new_data.append(w)
  return new_data

def remove_punc(data):
  new_data = []
  for w in data:
    if w not in string.punctuation:
      new_data.append(w)
  return new_data
'''
def stemming(data):
  new_data = []
  ps = PorterStemmer()
  for w in data:
    new_word = ps.stem(w)
    new_data.append(new_word)
  return new_data
'''

def lemmatizer(data):
  new_data = []
  lemm = WordNetLemmatizer()
  for w in data:
    new_word = lemm.lemmatize(w)
    new_data.append(new_word)
  return new_data

def preprocess(data):
  data = data.lower()
  data = word_tokenize(data)
  data = remove_stopwords(data)
  data = remove_punc(data)
  data = lemmatizer(data)
  return data

In [ ]:
def preprocess_data(raw_text1):
  text = []
  for sent in raw_text1:
    new_sent = preprocess(sent)
    new_sent = " ".join(new_sent) 
    text.append(new_sent)
  return text

In [ ]:
text = preprocess_data(raw_text)

import pickle
pickle.dump(text, open("/content/drive/MyDrive/preproc_text_PROJECT2","wb"))
pickle.dump(y, open("/content/drive/MyDrive/preproc_y_PROJECT2","wb"))
pickle.dump(app_num, open("/content/drive/MyDrive/preproc_appno_PROJECT2","wb"))

In [ ]:
#import pickle
#text = pickle.load(open("/content/drive/MyDrive/preproc_text_PROJECT2","rb"))
#y = pickle.load(open("/content/drive/MyDrive/preproc_y_PROJECT2","rb"))
#app_num = pickle.load(open("/content/drive/MyDrive/preproc_appno_PROJECT2","rb"))
#train_appno = np.array(app_num[:1350])
#test_appno = np.array(app_num[1350:])
#train_y = np.array(y[:1350])
#test_y = np.array(y[1350:])

In [ ]:
train_text = text[:1350]
test_text = text[1350:]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
Vect = TfidfVectorizer(ngram_range=(1,1), max_features=10000)
Vect.fit(train_text)
X = Vect.transform(train_text)
X = X.toarray()
#print(Vect.get_feature_names())

In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
mnb = MultinomialNB(alpha=0.5)
mnb.fit(X,train_y)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB(var_smoothing=1.0)
gnb.fit(X,train_y)

GaussianNB(priors=None, var_smoothing=1.0)

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
sgd.fit(X,train_y)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=None, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [ ]:
from sklearn.svm import LinearSVC
svmclf = LinearSVC(tol=1e-5)
svmclf.fit(X,train_y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=1e-05,
          verbose=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier(max_features='log2')
rfclf.fit(X, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.svm import SVC
svcclf = SVC()
svcclf.fit(X, train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlpclf = MLPClassifier(hidden_layer_sizes=(5000, 1000, 500), activation='relu', learning_rate_init=0.01)
mlpclf.fit(X, train_y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5000, 1000, 500), learning_rate='constant',
              learning_rate_init=0.01, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
import random
queries = ["nonconductive fibrous web substrate aqueous containing a conductive  wet add - on , and dried electrically conductive fabric",
           "tubing pressure , control line activation , shunt tube activation , and mechanical activation latching tool", 
           "variety of conductors , including silver - plated aluminum or copper . the filter housing 10 could also be constructed of , or coated with , a high temperature superconductor",
           "lumbar intervertebral expandable screw box 100 with two bdft screws ",
           "door frame upright at a point where it falls under the hand of the householder .pin attached ring 12",
           "typical optical transponder comprising a client receiver transmitter line transmitter and devices",
           "extrusion apparatus according to a preferred embodiment rotatable about its axis in the direction indicated by the arrow",
           "pharmacological activity as inhibitors of the enzyme phospholipase, arachidonic acid from phospholipids",
           "typical shape memory polymers incorporate hard and soft segments",
           "supporting an individual in a prone position with upper frame with a pair of forward , transversely spaced , downwardly extending legs",
           "air , nitrogen , carbon dioxide and mixtures thereof . the gas may be present in the fracturing fluid with stable foams",
           "teos in the deposition vessel  higher deposition rateof oxide and greater deposition thickness than non - doped silicon",
           "ethanol extractant liquid unsubstituted and substituted liquid secondary amines",
           "fast fluidized bed reactor",
           "rotatable roll jacket with antifriction bearing in a bearing bracket"]
queries_y = ['d','e','y','a','g','h','b','c','f','a','e','y','c','b','d']

for i in range(100):
  txt = test_text[i].split(" ")
  n = random.randint(6,18)
  start = random.randint(4,800)
  d1 = random.randint(start+1,start+n-2)
  d2 = random.randint(d1,start+n-1)
  q = txt[start:d1]+txt[d1+1:d2]+txt[d2+1:start+n]
  q = " ".join(q) 
  queries.append(q)
  queries_y.append(test_y[i])


In [ ]:
from sklearn.metrics import accuracy_score
queries = preprocess_data(queries)
#queries_y = test_y
query_X = Vect.transform(queries)
query_X = query_X.toarray()
query_X = Vect.transform(queries)
query_X = query_X.toarray()

nb_predict = gnb.predict(query_X)
mnb_predict = mnb.predict(query_X)
sgd_predict = sgd.predict(query_X)
lsvc_predict = svmclf.predict(query_X)
rfclf_predict = rfclf.predict(query_X)
svc_predict = svcclf.predict(query_X)
print("gaussian NB",accuracy_score(queries_y,nb_predict))
print("Multinomial NB",accuracy_score(queries_y,mnb_predict))
print("SGD",accuracy_score(queries_y,sgd_predict))
print("Linear SVC",accuracy_score(queries_y,lsvc_predict))
print("Random Forest",accuracy_score(queries_y,rfclf_predict))
print("SVM",accuracy_score(queries_y,svc_predict))

gaussian NB 0.26956521739130435
Multinomial NB 0.391304347826087
SGD 0.2956521739130435
Linear SVC 0.3217391304347826
Random Forest 0.0782608695652174
SVM 0.3130434782608696


In [ ]:
mlp_pred = mlpclf.predict(query_X)
print("MLP",accuracy_score(queries_y,mlp_pred))

MLP 0.20869565217391303


BM25

In [ ]:
!pip install rank_bm25

In [ ]:
patents = []
for i in range(len(text)):
  pat = [app_num[i],text[i]]
  patents.append(pat)  

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
Vect = TfidfVectorizer(ngram_range=(1,1), max_features=10000, max_df=1000)
Vect.fit(text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
query = "cloth drying machine"
#query = preprocess_data(query)
print(query)
from rank_bm25 import BM25Okapi
tokenized_text = [doc.split(" ") for doc in text]
tokenized_query = query.split(" ")
bm25 = BM25Okapi(tokenized_text)
doc_scores = bm25.get_scores(tokenized_query)
retrieved_docs = bm25.get_top_n(tokenized_query, patents, n=10)

cloth drying machine


In [ ]:
docs = []
for doc in retrieved_docs:
  print(doc)
  docs.append(doc[1])

['US-96926604-A', 'referring drawing fig1 show part whole cloth quilt 10 top fabric panel 12 meant generally viewed often bottom fabric panel 14 generally meant viewed le top panel intermediate batting 16 pattern stitch 18 forming sculptured effect 22 connecting three layer special care taken edge 28 quilt 10 insure surface panel 12 14 directly engaged along outer edge prevent batting visible edge insure batting panel 16 also secured surface panel scale 20 one inch increment shown appears quilt 10 fig1 indicate scale stitch pattern 18 top fabric panel 12 least perhaps bottom fabric panel 14 also dyed printed pigmented otherwise colored containing carrying solid color print pattern characteristic one surface fabric simply called colored vivid bright pale applied griege good applied yarn weaving printed dyed prepared print fabric quilt washed artificially faded inventive process explained later purpose invention achieve antiqued finish whole cloth quilt fabric antiqued effect worn faded 

In [ ]:
query_vec = Vect.transform([query])
query_vec = query_vec.toarray()

docs_vec = Vect.transform(docs)
docs_vec = docs_vec.toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

for vec in docs_vec:
  print(cosine_similarity(query_vec, [vec]))

[[0.08089284]]
[[0.50667065]]
[[0.35337601]]
[[0.32754831]]
[[0.12061753]]
[[0.26255299]]
[[0.16880851]]
[[0.21370825]]
[[0.06039437]]
[[0.23878558]]


In [ ]:
from sklearn.model_selection import GridSearchCV
mnb_clf = MultinomialNB()
#params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
params_multi = {'alpha':np.linspace(0.5,1.5,6)}
gs_NB = GridSearchCV(estimator=mnb_clf, 
                 param_grid=params_multi,
                 verbose=1, 
                 scoring='accuracy') 
gs_NB.fit(X, train_y)
print(gs_NB.best_params_)
print(gs_NB.best_score_)